In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score

In [2]:
df_bt = pd.read_csv('blocks_timestamps.csv') # Timestamps for each block
df_om = pd.read_csv('ordermatches_all.csv')  # Order details for each order (block)
df_pp = pd.read_csv('pepecash_prices.csv')   # USD value of pepecash over time (timeblock)
df_xp = pd.read_csv('xcp_prices.csv')        # USD value of xcp over time (timeblock)

In [17]:
#df_bt

In [18]:
#df_om

In [19]:
#df_pp

In [20]:
#df_xp

In [7]:
# Need to throw out all data that isn't xcp or pepecash value
# JK no we don't
# df_om.drop(df_om[df_om.ForwardAsset != ('PEPECASH' or 'XCP')].index, inplace=True)

In [8]:
#df_om_merge = pd.concat([df_om, df_bt], axis=1, join='inner')
df_om_merge = pd.merge(df_om, df_bt, on='Block', how='inner')

In [21]:
#df_om.insert
#df_om_merge = pd.merge(df_om, df_bt)
#df_om_merge = pd.concat([df_om, df_bt], on='Block', axis=1)
df_om_merge

,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,428940,RAREPEPE,1.0,XCP,2.00,1473403689
1,429221,SHITCOINCARD,230.0,XCP,13.80,1473561677
2,429221,RAREPEPE,10.0,XCP,15.00,1473561677
3,429228,SHITCOINCARD,3.0,XCP,0.18,1473567368
4,429228,RAREPEPE,1.0,XCP,1.50,1473567368
5,429398,SHITCOINCARD,12.0,XCP,0.72,1473654430
6,429398,RAREPEPE,2.0,XCP,3.00,1473654430
7,429403,RAREPEPE,1.0,XCP,1.50,1473656361
8,429403,SHITCOINCARD,755.0,XCP,45.30,1473656361
9,429466,RAREPEPE,1.0,XCP,1.50,1473687080


In [10]:
df_om_merge.columns

Index(['Block', 'ForwardAsset', 'ForwardQuantity', 'BackwardAsset',
       'BackwardQuantity', 'Timestamp'],
      dtype='object')

In [11]:
'''#for row in df_om_merge:
df_om_merge2 = df_om_merge
for x, y in df_om_merge2.iterrows():
    check = False
    i = 0
    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'XCP'):
        while check != True or i<519:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_xp.Price[i+1]
                df_om_merge2.BackwardAsset[x] = 'USD'
            i=i+1
        i=0
        check = False
    elif (df_om_merge2.loc[x, 'ForwardAsset'] == 'XCP'):
        while check != True or i<519:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_xp.Price[i+1]
                df_om_merge2.ForwardAsset[x] = 'USD'
            i=i+1
        i=0
        
    # PEPECASH Conversion
        check = False
    elif (df_om_merge2.loc[x, 'BackwardAsset'] == 'PEPECASH'):
        while check != True or i<490:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_pp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_pp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_pp.Price[i+1]
                df_om_merge2.BackwardAsset[x] = 'USD'
            i=i+1
        i=0
        check = False
    else:#(df_om_merge2.loc[x, 'ForwardAsset'] == 'PEPECASH'):
        while check != True or i<490:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_pp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_pp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_pp.Price[i+1]
                df_om_merge2.ForwardAsset[x] = 'USD'
            i=i+1
           
        
        #for label, row in df_xp.iterrows():
                #print(label)
                #print(row)
                #if(ts >= df_xp.Timestamp[x] and ts < df_xp.Timestamp[x+1]):
                    #print(x)
                    #df_om_merge.BackwardQuantity[i] = df_xp.Price[x] * df_om_merge.BackwardQuantity[i]
                    #df_om_merge.BackwardAsset = 'USD'
                
#print(df_om_merge.Timestamp)
'''
#Commented out because it takes forever

"#for row in df_om_merge:\ndf_om_merge2 = df_om_merge\nfor x, y in df_om_merge2.iterrows():\n    check = False\n    i = 0\n    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'XCP'):\n        while check != True or i<519:\n            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):\n                check = True\n                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_xp.Price[i+1]\n                df_om_merge2.BackwardAsset[x] = 'USD'\n            i=i+1\n        i=0\n        check = False\n    elif (df_om_merge2.loc[x, 'ForwardAsset'] == 'XCP'):\n        while check != True or i<519:\n            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):\n                check = True\n                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_xp.Price[i+1]\n               

In [28]:
# This dataset now has the USD equivalent of the XCP values and PEPECASH
#Load ordermatches_USD.csv because it is faster doing this
df_om_merge2 = pd.read_csv('ordermatches_USD.csv')   
df_om_merge2

,Unnamed: 0,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,0,428940,RAREPEPE,1.000000,USD,6.660000,1473403689
1,1,429221,SHITCOINCARD,230.000000,USD,44.988000,1473561677
2,2,429221,RAREPEPE,10.000000,USD,48.900000,1473561677
3,3,429228,SHITCOINCARD,3.000000,USD,0.586800,1473567368
4,4,429228,RAREPEPE,1.000000,USD,4.890000,1473567368
5,5,429398,SHITCOINCARD,12.000000,USD,2.491200,1473654430
6,6,429398,RAREPEPE,2.000000,USD,10.380000,1473654430
7,7,429403,RAREPEPE,1.000000,USD,5.190000,1473656361
8,8,429403,SHITCOINCARD,755.000000,USD,156.738000,1473656361
9,9,429466,RAREPEPE,1.000000,USD,5.655000,1473687080


In [13]:
'''df_om_merge2 = pd.read_csv('ordermatches_USD.csv')
for x, y in df_om_merge2.iterrows():
    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'USD'):
        df_om_merge2.ForwardQuantity[x], df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x], df_om_merge2.ForwardQuantity[x]
'''
#Swap quantities - dollar on right, card on left
#Commented out because it takes forever

"df_om_merge2 = pd.read_csv('ordermatches_USD.csv')\nfor x, y in df_om_merge2.iterrows():\n    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'USD'):\n        df_om_merge2.ForwardQuantity[x], df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x], df_om_merge2.ForwardQuantity[x]\n"

In [14]:
'''for x, y in df_om_merge2.iterrows():
    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'USD'):
        df_om_merge2.ForwardAsset[x], df_om_merge2.BackwardAsset[x] = df_om_merge2.BackwardAsset[x], df_om_merge2.ForwardAsset[x]
'''
#Swap assets - same as above

"for x, y in df_om_merge2.iterrows():\n    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'USD'):\n        df_om_merge2.ForwardAsset[x], df_om_merge2.BackwardAsset[x] = df_om_merge2.BackwardAsset[x], df_om_merge2.ForwardAsset[x]\n"

In [29]:
#df_om_merge2
df_om_merge2.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\Ordered.csv')
df_om_merge3 = pd.read_csv('Ordered.csv')  
df_om_merge3.drop(df_om_merge2.columns[df_om_merge2.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_om_merge3


,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,428940,RAREPEPE,1.0,USD,6.660000,1473403689
1,429221,SHITCOINCARD,230.0,USD,44.988000,1473561677
2,429221,RAREPEPE,10.0,USD,48.900000,1473561677
3,429228,SHITCOINCARD,3.0,USD,0.586800,1473567368
4,429228,RAREPEPE,1.0,USD,4.890000,1473567368
5,429398,SHITCOINCARD,12.0,USD,2.491200,1473654430
6,429398,RAREPEPE,2.0,USD,10.380000,1473654430
7,429403,RAREPEPE,1.0,USD,5.190000,1473656361
8,429403,SHITCOINCARD,755.0,USD,156.738000,1473656361
9,429466,RAREPEPE,1.0,USD,5.655000,1473687080


In [30]:
#df_om_merge3['PricePerPepe'] = df_om_merge3['BackwardQuantity'] / df_om_merge3['ForwardQuantity']
#df_om_merge3.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\PPP.csv')
df_om_merge4 = pd.read_csv('PPP.csv') 
df_om_merge4
#Commented out because it takes a while, load from PPP to continue

,Unnamed: 0,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp,PricePerPepe
0,0,428940,RAREPEPE,1.0,USD,6.660000,1473403689,6.660000
1,1,429221,SHITCOINCARD,230.0,USD,44.988000,1473561677,0.195600
2,2,429221,RAREPEPE,10.0,USD,48.900000,1473561677,4.890000
3,3,429228,SHITCOINCARD,3.0,USD,0.586800,1473567368,0.195600
4,4,429228,RAREPEPE,1.0,USD,4.890000,1473567368,4.890000
5,5,429398,SHITCOINCARD,12.0,USD,2.491200,1473654430,0.207600
6,6,429398,RAREPEPE,2.0,USD,10.380000,1473654430,5.190000
7,7,429403,RAREPEPE,1.0,USD,5.190000,1473656361,5.190000
8,8,429403,SHITCOINCARD,755.0,USD,156.738000,1473656361,0.207600
9,9,429466,RAREPEPE,1.0,USD,5.655000,1473687080,5.655000


In [33]:
#df_om_merge2.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\PPP.csv')
df_om_merge4 = pd.read_csv('PPP.csv')  
df_om_merge4.drop(df_om_merge4.columns[df_om_merge4.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_om_merge4.rename(columns= {'ForwardAsset': 'CardName',
                               'ForwardQuantity': 'NumCards',
                               'BackwardAsset': 'Payment',
                               'BackwardQuantity': 'PriceUSD'}, inplace=True)
df_om_merge4 = df_om_merge4.reindex(columns=['Block', 'Timestamp', 'CardName', 'NumCards', 'Payment',
                                            'PriceUSD', 'PricePerPepe'])
df_om_merge4
#df_om_merge4.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\Cleanup.csv')

#change column names

,Block,Timestamp,CardName,NumCards,Payment,PriceUSD,PricePerPepe
0,428940,1473403689,RAREPEPE,1.0,USD,6.660000,6.660000
1,429221,1473561677,SHITCOINCARD,230.0,USD,44.988000,0.195600
2,429221,1473561677,RAREPEPE,10.0,USD,48.900000,4.890000
3,429228,1473567368,SHITCOINCARD,3.0,USD,0.586800,0.195600
4,429228,1473567368,RAREPEPE,1.0,USD,4.890000,4.890000
5,429398,1473654430,SHITCOINCARD,12.0,USD,2.491200,0.207600
6,429398,1473654430,RAREPEPE,2.0,USD,10.380000,5.190000
7,429403,1473656361,RAREPEPE,1.0,USD,5.190000,5.190000
8,429403,1473656361,SHITCOINCARD,755.0,USD,156.738000,0.207600
9,429466,1473687080,RAREPEPE,1.0,USD,5.655000,5.655000


In [34]:
df_om_merge5 = df_om_merge4
df_om_merge5.sort_values(["CardName", "Timestamp"], axis=0, inplace=True)
df_om_merge5.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\Sorted.csv')
df_om_merge4
#sort by cardname, then timestamp

,Block,Timestamp,CardName,NumCards,Payment,PriceUSD,PricePerPepe
1396,434445,1476546441,BACHELORPEPE,1.0,USD,17.040000,17.040000
1783,434852,1476797843,BACHELORPEPE,1.0,USD,12.450000,12.450000
3349,439210,1479306308,BACHELORPEPE,1.0,USD,8.840000,8.840000
4817,446214,1483329596,BACHELORPEPE,1.0,USD,38.400000,38.400000
10456,455933,1488760516,BACHELORPEPE,1.0,USD,29.000000,29.000000
17129,470549,1497035519,BACHELORPEPE,1.0,USD,261.060000,261.060000
19864,476883,1500657195,BACHELORPEPE,1.0,USD,287.700000,287.700000
19865,476883,1500657195,BACHELORPEPE,1.0,USD,287.700000,287.700000
20900,477251,1500852631,BACHELORPEPE,1.0,USD,333.420000,333.420000
22441,480668,1502812058,BACKTOPEPE,5.0,USD,25.055420,5.011084


In [ ]:
len(df_om_merge4['CardName'].unique())
#check how many unique cards there are

In [ ]:
#TODO:
#Come up with plan B
#Otherwise, see if it is a good deal or not